# Goals

#### Predict Gender and age in given groups based on the datasets provided 
These datasets come in csv files connected by different identification keys. 

# Understand the data
This is crucial! There are basic explanations on the competitions [website](https://www.kaggle.com/c/talkingdata-mobile-user-demographics/data). Here is the data connection diagramm given:
![Data diagramm](data_tree.png)

We also already now that the company names are provided in chinese, which is not particularly helpful. Then, we see that the apps already are categorised, which is handy. However, doubt is present that the variable can be used out-of-the box.

In [1]:
# Set input dir
import sys
# perform imports so that we can use single cells w/o worrying about imports
import pandas as pd
from pandas import DataFrame as df
import IPython as ip

import seaborn as sns
sns.set(color_codes=True)

%matplotlib inline
import matplotlib.pyplot as plt

sys.path.append("/home/mschlupp/pythonTools")
tmp = %pwd
files_dir = tmp + "/files" 

#### Load some nice colors from [my github](https://github.com/mschlupp/pythonTools/tree/master/plotting).
Then delete a few lines of the original file to make the code a bit leaner.

In [4]:
# %load '/home/mschlupp/pythonTools/plotting/nice_colors.py'

'''
Color dictionaries from 
http://tableaufriction.blogspot.de/2012/11/finally-you-can-use-tableau-data-colors.html
'''

Tableau10_Medium = { 
	"blue"   : (114/255.,158/255.,206/255.),
	"orange" : (255/255.,158/255.,74/255.),
	"green"  : (103/255.,191/255.,92/255.),
	"red"    : (237/255.,102/255.,93/255.),
	"purple" : (173/255.,139/255.,201/255.),
	"brown"  : (168/255.,120/255.,110/255.),
	"pink"   : (237/255.,151/255.,202/255.),
	"gray"   : (162/255.,162/255.,162/255.),
	"olive"  : (205/255.,204/255.,93/255.),
	"cyan"   : (109/255.,204/255.,218/255.)
}

ColorBlind10 = {
	"azure"        : (0./255.,107./255.,164/255.),
	"orange"       : (255./255.,128./255.,14/255.),
	"grey_medium"  : (171./255.,171./255.,171/255.),
	"grey_dark"    : (89./255.,89./255.,89/255.),
	"blue_medium"  : (95./255.,158./255.,209/255.),
	"orange_dark"  : (200./255.,82./255.,0/255.),
	"grey"         : (137./255.,137./255.,137/255.),
	"blue_light"   : (162./255.,200./255.,236/255.),
	"orange_light" : (255./255.,188./255.,121/255.),
	"grey_light"   : (207./255.,207./255.,207/255.)
}

We left of the first notebook, `1_understand_data`, with a merged data file. We will load it first.

In [8]:
traintest_phone = pd.read_csv(files_dir+'/traintest_phone.csv')
traintest_phone.head(2)

,Unnamed: 0,age,device_id,gender,group,isTrain,phone_brand,device_model
0,0,35,-8076087639492063270,M,M32-38,1,小米,MI 2
1,1,35,-2897161552818060146,M,M32-38,1,小米,MI 2
